In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import block_diag
import itertools
from timeit import default_timer as timer
import scipy as sc
import pickle
import scipy.sparse as scs
from functools import lru_cache, cache
from joblib import Parallel, delayed


In [2]:
def boolean_combinations(n):
    return [
        *itertools.product(
            *[[0, 1, 2, 3] for _ in range(n)]
    )]

In [3]:
#inverted number of rishons of each link configuration
#0=Left, 1=Right, 2=Down

s0 = np.array([0,0,0])
s1 = np.array([1,1,0])
s2 = np.array([1,0,1])
s3 = np.array([0,1,1])

r = np.array([s0,s1,s2,s3]) #rishon register

In [4]:
def ten2four(n):
    n = n % (4294967296)
    s = np.base_repr(n,4)
    return np.array(list(s.zfill(16)),dtype=int)

In [5]:
factor_basis = np.array(boolean_combinations(8))

b_factor = np.array(boolean_combinations(11))

s_factor = np.array(boolean_combinations(5))

In [6]:
def is_gauß2(state):
    c = np.zeros((9,4**11),dtype=bool)
    c[0] = (r[state[:,0],1]+r[state[:,1],2]+r[state[:,5],0])%2 == 0
    c[1] = (r[state[:,1],1]+r[state[:,2],2]+r[state[:,6],0])%2 == 0
    c[2] = (r[state[:,2],1]+r[state[:,3],2]+r[state[:,7],0])%2 == 0
                    
    c[3] = (r[state[:,4],1]+r[state[:,5],2]+r[state[:,9],0])%2 == 0
    c[4] = (r[state[:,5],1]+r[state[:,6],2]+r[state[:,10],0])%2 == 0
    c[5] = (r[state[:,6],1]+r[state[:,7],2]+r[state[:,11],0])%2 == 0

    c[6] = (r[state[:,8],1]+r[state[:,9],2]+r[state[:,13],0])%2 == 0
    c[7] = (r[state[:,9],1]+r[state[:,10],2]+r[state[:,14],0])%2 == 0
    c[8] = (r[state[:,10],1]+r[state[:,11],2]+r[state[:,15],0])%2 == 0

    return np.all(c,axis=0)

In [7]:
key = np.array([4**7,4**6,4**5,4**4,4**3,4**2,4**1,4**0])

In [8]:
def basis_gen(i):
    state = s_factor[i]
    temp = is_gauß2(np.hstack((np.repeat(state[:, np.newaxis], 4**11, axis=1).T,b_factor)))
    sum = np.sum(temp)
    if sum>0:
        return np.ones(sum)*4194304*i+np.argwhere(temp)[:,0]


In [9]:
results = Parallel(n_jobs=14)(delayed(basis_gen)(i) for i in range(4**5))
results_clean = [i for i in results if i is not None]
h_array = np.array(results_clean, dtype=int)
h_array = h_array.flatten()
states = [ten2four(i) for i in h_array]

In [ ]:
np.savez_compressed("gauge_indices_4x4_sort1",h_array)